### Install some libs

!pip install treelib

!pip3 install tree_sitter

### Import some libs

In [1]:
from treelib import Node, Tree
from tree_sitter import Language, Parser
import os
import numpy as np

### Initialize Tree-sitter go parser

In [2]:
Language.build_library(
  # Store the library in the `build` directory
  'build/my-languages.so',

  # Include one or more languages
  [
    'tree-sitter-go'
  ]
)

GO_LANGUAGE = Language('build/my-languages.so', 'go')

parser = Parser()
parser.set_language(GO_LANGUAGE)

## Step I : Find the Feature Toggles (FT)

```
$ egrep -R "=" loomchain/features/features.go | grep -v "//"
py> keys = [k.split('=')[0].split(' ')[0] for k in feats.replace('\t', '').split('\n')]
```

In [3]:
keywords = ['TGCheckTxHashFeature',
 'TGHotWalletFeature',
 'TGCheckZeroAmount',
 'TGFixERC721Feature',
 'TGBinanceContractMappingFeature',
 'TGWithdrawalLimitFeature',
 'TGVersion1_1',
 'TGVersion1_2',
 'TGVersion1_3',
 'TGVersion1_4',
 'TGVersion1_5',
 'TGVersion1_6',
 'TGVersion1_7',
 'AddressMapperVersion1_1',
 'AuthSigTxFeaturePrefix',
 'MultiChainSigTxMiddlewareVersion1_1',
 'DPOSVersion3Feature',
 'DPOSVersion3_1',
 'DPOSVersion3_2',
 'DPOSVersion3_3',
 'DPOSVersion3_4',
 'DPOSVersion3_5',
 'DPOSVersion3_6',
 'DPOSVersion3_7',
 'DPOSVersion3_8',
 'DPOSVersion3_9',
 'DPOSVersion3_10',
 'DPOSVersion2_1',
 'EvmTxReceiptsVersion2Feature',
 'EvmTxReceiptsVersion3',
 'EvmTxReceiptsVersion3_1',
 'EvmTxReceiptsVersion3_2',
 'EvmTxReceiptsVersion3_3',
 'EvmTxReceiptsVersion3_4',
 'DeployerWhitelistFeature',
 'UserDeployerWhitelistFeature',
 'UserDeployerWhitelistVersion1_1Feature',
 'UserDeployerWhitelistVersion1_2Feature',
 'MigrationTxFeature',
 'MigrationTxVersion1_1Feature',
 'MigrationFeaturePrefix',
 'ChainCfgVersion1_1',
 'ChainCfgVersion1_2',
 'ChainCfgVersion1_3',
 'ChainCfgVersion1_4',
 'EthTxFeature',
 'EvmDBFeature',
 'CoinVersion1_1Feature',
 'CoinVersion1_2Feature',
 'CoinVersion1_3Feature',
 'AuxEvmDBFeature',
 'AppStoreVersion3_1',
 'DeployTxVersion1_1Feature',
 'CheckTxValueFeature',
 'EvmConstantinopleFeature']

### List files with keywords

#### List all .go files

In [4]:
go_folders = [x[0] for x in os.walk("./loomchain/")]

go_files = []
for dir_name in go_folders:
    files = [dir_name+"/"+k for k in os.listdir(dir_name) if k[len(k)-3:] ==".go"]
    go_files.extend(files)

print("10 random .go files in this project :")
for k in range(10):
    print(go_files[np.random.randint(len(go_files))])

10 random .go files in this project :
./loomchain/db/metrics/goleveldb_stats_collector.go
./loomchain/builtin/plugins/ethcoin/eth_coin_test.go
./loomchain/rpc/eth/jsonrpc_conversion.go
./loomchain/rpc/eth_rpc_funcs_test.go
./loomchain/evm/interfaces.go
./loomchain/store/evmstore_test.go
./loomchain/rpc/mock_query_server.go
./loomchain/cmd/loom/sendtx.go
./loomchain/evm/config.go
./loomchain/registry/v1/registry_test.go


In [5]:
len(go_files)

325

#### List the interesting files, containing at least one keyword with the featureflag 

In [6]:
go_file_interests = []

for file in go_files:
    s = ""
    with open(file, "r") as f:
        s+=f.read()+"\n"
    kw_file = [k for k in keywords if k in s]
    if len(kw_file) > 0 and "FeatureEnabled" in s:
        print(file, "contains :")
        for kw in kw_file:
            print("->",kw)
        go_file_interests.append(file)
        print("\n")

./loomchain//app.go contains :
-> EvmTxReceiptsVersion3
-> EvmTxReceiptsVersion3_1


./loomchain/plugin/chainconfig_manager.go contains :
-> ChainCfgVersion1_3
-> ChainCfgVersion1_4


./loomchain/plugin/validators_manager_v3.go contains :
-> DPOSVersion3_2


./loomchain/auth/multi_chain_sigtx_middleware.go contains :
-> AuthSigTxFeaturePrefix
-> MultiChainSigTxMiddlewareVersion1_1
-> EthTxFeature


./loomchain/auth/chainconfig_middleware.go contains :
-> AuthSigTxFeaturePrefix


./loomchain/vm/handler.go contains :
-> DeployTxVersion1_1Feature


./loomchain/tx_handler/migration_tx_handler.go contains :
-> MigrationTxFeature
-> MigrationTxVersion1_1Feature
-> MigrationFeaturePrefix


./loomchain/tx_handler/eth_tx_handler.go contains :
-> EthTxFeature


./loomchain/throttle/deployer-middleware.go contains :
-> DeployerWhitelistFeature
-> UserDeployerWhitelistFeature
-> EthTxFeature


./loomchain/builtin/plugins/user_deployer_whitelist/user_deployer_whitelist.go contains :
-> UserDeployer

In [7]:
len(go_file_interests)

23

## Step 3 - Search Feature Toggles dependencies in the AST

In [8]:
for gfi in go_file_interests:
    
    s = ""
    with open(gfi, "r") as f:
        s+=f.read()+"\n"

    print("File :            ", gfi, "\n")

    source = bytes(s, "utf8")
    ast = parser.parse(source)

    root_node = ast.root_node

    tree = Tree()

    root_node = ast.root_node

    type_nodes = dict()

    def get_name(node):
        return source[node.start_byte:node.end_byte].decode('utf8')

    def get_id(node):
        global type_nodes
        node_type = node.type
        if node_type not in type_nodes:
            type_nodes[node_type]=1
        else:
            type_nodes[node_type]+=1
        return node_type+str(type_nodes[node_type])

    def process(root_id, node):
        global tree
        node_id = get_id(node)
        node_name = get_name(node)
        for kw in keywords:
            if kw in node_name and node.type == 'if_statement':
                print("\n\n"+get_name(node)+"\n\n")
                for c in node.children:
                    if c.type == 'binary_expression' or c.type == 'unary_expression' or c.type == 'call_expression':
                        print("->",get_name(c))
        tree.create_node(node_name,
                         node_id,
                         parent = root_id)
        if len(node.children) != 0:
            for i in range(len(node.children)):
                process(node_id, node.children[i])

    tree.create_node("root", "root")
    for i in range(len(root_node.children)):
        process("root", root_node.children[i])
    print("\n")

    #tree.show()

File :             ./loomchain//app.go 



if state.FeatureEnabled(features.EvmTxReceiptsVersion3_1, false) {
		r = a.deliverTx2(storeTx, txBytes)
	} else {
		r = a.deliverTx(storeTx, txBytes)
	}


-> state.FeatureEnabled(features.EvmTxReceiptsVersion3_1, false)


if state.FeatureEnabled(features.EvmTxReceiptsVersion3_1, false) {
		r = a.deliverTx2(storeTx, txBytes)
	} else {
		r = a.deliverTx(storeTx, txBytes)
	}


-> state.FeatureEnabled(features.EvmTxReceiptsVersion3_1, false)


if !isCheckTx {
		a.EventHandler.Commit(uint64(a.curBlockHeader.GetHeight()))

		saveEvmTxReceipt := r.Info == utils.CallEVM || r.Info == utils.DeployEvm ||
			state.FeatureEnabled(features.EvmTxReceiptsVersion3, false) || a.ReceiptsVersion == 3

		if saveEvmTxReceipt {
			if err := a.EventHandler.LegacyEthSubscriptionSet().EmitTxEvent(r.Data, r.Info); err != nil {
				log.Error("Emit Tx Event error", "err", err)
			}

			reader := a.ReceiptHandlerProvider.Reader()
			if reader.GetCurrentReceipt() != nil {
	



File :             ./loomchain/builtin/plugins/chainconfig/chainconfig.go 



if !ctx.FeatureEnabled(features.ChainCfgVersion1_3, false) {
		return ErrFeatureNotEnabled
	}


-> !ctx.FeatureEnabled(features.ChainCfgVersion1_3, false)


if ctx.FeatureEnabled(features.DPOSVersion3Feature, false) {
		contractAddr, err := ctx.Resolve("dposV3")
		if err != nil {
			return nil, errors.Wrap(err, "failed to resolve address of DPOSv3 contract")
		}

		req := &dposv3.ListValidatorsRequest{}
		var resp dposv3.ListValidatorsResponse
		if err := contract.StaticCallMethod(ctx, contractAddr, "ListValidators", req, &resp); err != nil {
			return nil, errors.Wrap(err, "failed to call ListValidators")
		}

		validators := make([]loom.Address, 0, len(resp.Statistics))
		for _, v := range resp.Statistics {
			if v != nil {
				addr := loom.UnmarshalAddressPB(v.Address)
				validators = append(validators, addr)
			}
		}
		return validators, nil
	}


-> ctx.FeatureEnabled(features.DPOSVersion3Feature, fals



File :             ./loomchain/builtin/plugins/address_mapper/address_mapper.go 



if ctx.FeatureEnabled(features.AddressMapperVersion1_1, false) {
		allowedSigTypes = append(allowedSigTypes, evmcompat.SignatureType_BINANCE)
	}


-> ctx.FeatureEnabled(features.AddressMapperVersion1_1, false)


File :             ./loomchain/builtin/plugins/coin/coin.go 



if ctx.FeatureEnabled(features.CoinVersion1_2Feature, false) && req.Amount == nil {
		return ErrInvalidRequest
	}


-> ctx.FeatureEnabled(features.CoinVersion1_2Feature, false) && req.Amount == nil


if ctx.FeatureEnabled(features.CoinVersion1_3Feature, false) {
		gatewayAddr, err = ctx.Resolve("binance-gateway")
		if err == nil && minter.Compare(gatewayAddr) == 0 {
			return nil
		}
	}


-> ctx.FeatureEnabled(features.CoinVersion1_3Feature, false)


if ctx.FeatureEnabled(features.TGVersion1_7, false) {
		gatewayAddr, err = ctx.Resolve("bsc-gateway")
		if err == nil && minter.Compare(gatewayAddr) == 0 {
			return nil
		}
	}


-> c



File :             ./loomchain/cmd/loom/loom.go 



if cfg.DPOSVersion == 3 || state.FeatureEnabled(features.DPOSVersion3Feature, false) {
			createDPOSV3Ctx := getContractCtx("dposV3", vmManager)
			dposV3Ctx, err := createDPOSV3Ctx(state)
			if err != nil {
				return nil, err
			}
			validators, err := dposv3.ValidatorList(dposV3Ctx)
			if err != nil {
				return nil, err
			}
			return loom.NewValidatorSet(validators...), nil
		} else if cfg.DPOSVersion == 2 {
			createDPOSV2Ctx := getContractCtx("dposV2", vmManager)
			dposV2Ctx, err := createDPOSV2Ctx(state)
			if err != nil {
				return nil, err
			}
			validators, err := dposv2.ValidatorList(dposV2Ctx)
			if err != nil {
				return nil, err
			}
			return loom.NewValidatorSet(validators...), nil
		}


-> cfg.DPOSVersion == 3 || state.FeatureEnabled(features.DPOSVersion3Feature, false)


if cfg.DPOSVersion == 3 || state.FeatureEnabled(features.DPOSVersion3Feature, false) {
			return plugin.NewValidatorsManagerV3(pvm.(*plugin.P